In [5]:
!pip install duckdb

     ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
      --------------------------------------- 0.2/9.6 MB 5.3 MB/s eta 0:00:02
     - -------------------------------------- 0.4/9.6 MB 6.7 MB/s eta 0:00:02
     --- ------------------------------------ 0.8/9.6 MB 6.2 MB/s eta 0:00:02
     ---- ----------------------------------- 1.1/9.6 MB 6.6 MB/s eta 0:00:02
     ----- ---------------------------------- 1.4/9.6 MB 6.6 MB/s eta 0:00:02
     ------ --------------------------------- 1.6/9.6 MB 6.5 MB/s eta 0:00:02
     ------- -------------------------------- 1.9/9.6 MB 6.3 MB/s eta 0:00:02
     --------- ------------------------------ 2.2/9.6 MB 6.7 MB/s eta 0:00:02
     ---------- ----------------------------- 2.5/9.6 MB 6.7 MB/s eta 0:00:02
     ----------- ---------------------------- 2.8/9.6 MB 6.7 MB/s eta 0:00:02
     ------------ --------------------------- 3.1/9.6 MB 6.5 MB/s eta 0:00:01
     ------------- -------------------------- 3.3/9.6 MB 6.5 MB/s eta 0

In [6]:
# Importing necessary libraries and establishing a DuckDB connection

import pandas as pd
import numpy as np
import duckdb
import time
con = duckdb.connect(database=':memory:', read_only=False)

## Exploration Query

In [7]:
# Counting events by type from the optimized data

%%time
con.query('''
SELECT event_type,
       count(*) as event_count
FROM '../Data/optimised_raw_data.parquet'
GROUP BY 1''').df()

CPU times: total: 2.66 s
Wall time: 469 ms


,event_type,event_count
0,cart,2641249
1,purchase,835007
2,view,52490785


In [8]:
# Displaying 5 rows where the event type is 'view'

con.query('''
SELECT *
FROM '../Data/optimised_raw_data.parquet'
WHERE event_type = 'view'
LIMIT 5''').df()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-01-01 00:00:00 UTC,view,1005073,2232732093077520756,construction.tools.light,samsung,1130.020020,519698804,69b5d72f-fd6e-4fed-aa23-1286b2ca89a0
1,2020-01-01 00:00:01 UTC,view,1005192,2232732093077520756,construction.tools.light,meizu,205.669998,527767423,7f596032-ccbf-4643-9bad-e36a209512b4
2,2020-01-01 00:00:01 UTC,view,100063693,2053013552427434207,apparel.shirt,turtle,136.429993,519046195,d1e2f343-84bb-49bd-b13d-ca0f1ed9910e
3,2020-01-01 00:00:01 UTC,view,5100816,2232732103831716449,apparel.shoes,xiaomi,29.950001,518269232,0444841c-38ef-410c-b11f-7b35ea4e5991
4,2020-01-01 00:00:02 UTC,view,100014325,2232732103294845523,apparel.shoes.step_ins,intel,167.199997,587748686,31b7d4cf-dfac-4895-9927-90fa3254f860


In [9]:
# Counting distinct products, users, and sessions for 'view' events

%%time
con.query('''
SELECT 
COUNT(distinct product_id) as product_cnt_30days,
COUNT(distinct user_id) as user_cnt_30days,
COUNT(distinct user_session) as user_session_cnt_30days
FROM '../Data/optimised_raw_data.parquet' WHERE event_type='view' ''').df()

CPU times: total: 1min 3s
Wall time: 13.5 s


,product_cnt_30days,user_cnt_30days,user_session_cnt_30days
0,226209,4383932,13757026


### Data Null Check on user_session

In [10]:
# Counting rows with NULL values in key columns for 'view' events

con.query('''
SELECT COUNT(*) as null_cnt
FROM '../Data/optimised_raw_data.parquet'
WHERE event_type = 'view'
  AND (user_session IS NULL
       OR user_id IS NULL
       OR product_id IS NULL)''').df()

,null_cnt
0,0


### Check how many distinct PDP view event is performed per session?

In [11]:
# Counting unique product views per session for 'view' events

con.query('''
SELECT 
user_session,
COUNT(distinct product_id) as count_unique_product_views_per_session
FROM '../Data/optimised_raw_data.parquet' 
WHERE event_type='view' 
GROUP BY 1''').df()

,user_session,count_unique_product_views_per_session
0,305c943a-b89a-46a0-b1b3-79f94c2513cc,12
1,e9acc590-f9e7-463b-875d-bd9c94c5c26e,1
2,49bb8625-acec-4dcb-9df6-b099d92f794c,3
3,a0888180-5c58-437d-918e-c7be00e6cd6d,25
4,4e35f2c3-666c-4bc4-acc0-1dcfcfc78d65,21
...,...,...
13757021,d6b793e3-3f94-4268-838d-652475be7cdf,1
13757022,3ba7bb87-48f3-4e67-8608-57cdd16b2fdf,1
13757023,7480adac-d54d-427f-98aa-b9db5ee91674,1
13757024,eb6ba2e2-c66b-4671-a2f2-5a22cc6dfc52,1


In [12]:
# Calculating average unique product views per session for 'view' events

con.query('''
SELECT AVG(count_unique_product_views_per_session) as avg_unique_product_views_per_session  FROM (SELECT 
user_session,
COUNT(distinct product_id) as count_unique_product_views_per_session
FROM '../Data/optimised_raw_data.parquet' 
WHERE event_type='view' 
GROUP BY 1 ) as processed_table''').df()

,avg_unique_product_views_per_session
0,2.622409


OBSV: In one user session on an average users visit 2 to 3 distinct PDPs.

### Count total events with non-null sessions

In [13]:
# Counting 'view' events with non-null user sessions

con.query('''
SELECT COUNT(*) AS view_cnt
FROM '../Data/optimised_raw_data.parquet'
WHERE event_type = 'view'
  AND user_session IS NOT NULL''').df()

,view_cnt
0,52490785


### EDA on user_sessions

In [14]:
# Counting unique user sessions for 'view' events

con.query('''
SELECT count(DISTINCT user_session) as user_session_cnt
FROM '../Data/optimised_raw_data.parquet' where event_type = 'view' ''').df()

,user_session_cnt
0,13757026


In [15]:
# Repeating the count of unique product views per session for 'view' events

con.query('''
SELECT 
user_session,
COUNT(distinct product_id) as count_unique_product_views_per_session
FROM '../Data/optimised_raw_data.parquet' 
WHERE event_type='view' 
GROUP BY 1
''').df()

,user_session,count_unique_product_views_per_session
0,8eaaadb2-2150-4ef8-91ef-e058e8335d5c,11
1,bbd19bd8-e15a-4579-93d1-c597a6f3e6dd,10
2,3504cea9-c92d-4fc0-adeb-1f8b2fb92a9b,1
3,6d4383e2-9305-4a88-b062-8ff924f87f0b,7
4,376599fc-d9a5-429f-a12e-5e0603b894f2,2
...,...,...
13757021,2278e9de-184d-42d5-aef8-b454fd4da87c,1
13757022,c7fbaf2a-8e30-4b0c-85f2-e406454717b1,1
13757023,a615975c-7bde-4df3-8240-f00d8349a763,1
13757024,412579b3-f36b-4c02-9ae9-e42a76d42edb,1


In [16]:
# Filtering sessions with more than one unique product view for 'view' events

con.query('''
SELECT 
user_session,
COUNT(distinct product_id) as count_unique_product_views_per_session
FROM '../Data/optimised_raw_data.parquet' 
WHERE event_type='view' 
GROUP BY 1
HAVING COUNT(distinct product_id)>1
''').df()

,user_session,count_unique_product_views_per_session
0,c555c40b-db6a-44dc-b2f1-1789f869c6e4,11
1,721e0562-e49d-4b6c-8798-2cb6b8278bcc,4
2,6c3650df-e64c-f7d6-4134-125f55d0f872,7
3,d222821a-bbdf-4683-974d-387e69fabda0,6
4,4c42f5f6-2e77-414e-8a2e-e6c1b0395e49,4
...,...,...
5340110,b7495102-d8d0-4148-b91d-e47c53250f4e,3
5340111,6917046d-0f8a-4844-b2a9-3d40685d8e8f,2
5340112,d65e0b46-c0e1-43e8-8c0c-21e4d531b0bd,2
5340113,5850bf85-4706-4c84-a754-b20b4a6e2e93,2


## Observations:
* Number of entries for event_type = 'view': 52M+
* On an average, users on Flipkart visit 2.6 (roughly 3) distinct PDPs per session
* In month of Jan 2020, 13M+ session happened on Flipkart where users visited atleast 1 PDP
* 5M+ session had more than 1 distinct PDP view _i.e Product Detail Page_